Torch implementation of NMF - When done should be moved to a .py file to allow easy implementation

In [104]:
import torch
import numpy as np

from data import X

Non negative matrix factorization works by finding matrices A and B such that X = AB

In [138]:
class torchNMF(torch.nn.Module):
    def __init__(self, X, rank):
        super(torchNMF, self).__init__()
        
        #Shape of Matrix for reproduction
        n_row, n_col = X.shape
        self.X = torch.tensor(X)
        
        self.softplus = torch.nn.Softplus()
        
        #Initialization of Tensors/Matrices a and b with size NxR and RxM
        self.A = torch.nn.Parameter(torch.rand(n_row, rank, requires_grad=True))
        self.A = torch.nn.Parameter(self.softplus(self.A))
        
        self.B = torch.nn.Parameter(torch.rand(rank, n_col, requires_grad=True))
        self.B = torch.nn.Parameter(self.softplus(self.B))
        
    def forward(self):
        
        #Implementation of NMF - F(A, B) = ||X - AB||^2
        self.AB = torch.matmul(self.A, self.B)
        x = self.X - self.AB
        
        return x

In [139]:
#Defining frobenius Loss
class frobeniusLoss(torch.nn.Module):
    def __init__(self):
        super(frobeniusLoss, self).__init__()
        self.loss = torch.linalg.matrix_norm
    
    def forward(self, input):
        return self.loss(input, ord='fro')

In [ ]:
nmf = torchNMF(X, 10)

batch_size = 10

#optimizer for modifying learning rate, ADAM chosen because of https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
optimizer = torch.optim.Adam(nmf.parameters(), lr=0.1)

for epoch in range(200):
    
    #zero optimizer gradient
    optimizer.zero_grad()

    #forward
    output = nmf()
    
    #backward
    loss = frobeniusLoss()
    loss = loss.forward(output)
    loss.backward()
    
    #Update A and B
    optimizer.step()
        
    print(loss.item())
        